# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will use the dataset 'Healthcare For All' building a model to predict who will donate (TargetB) and how much they will give (TargetD). You will be using files_for_lab/learningSet.csv file which you have already downloaded from class.

## Scenario
You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

## Instructions
In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.

- Read that data into Python and call the dataframe donors.

- Check the datatypes of all the columns in the data.

- Check for null values in the dataframe. Replace the null values using the methods learned in class.

- Split the data into numerical and catagorical. Decide if any columns need their dtype changed.

    - Split the data into a training set and a test set.
    - Scale the features either by using normalizer or a standard scaler.
    - Encode the categorical features using One-Hot Encoding or Ordinal Encoding
    - Fit a logistic regression model on the training data.
    - Check the accuracy on the test data.
    
Note: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model has changed.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import time
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

## Importing pre-cleaned data and final cleaning

In [2]:
numerical = pd.read_csv('numerical7_02.csv')
categorical = pd.read_csv('categorical7_02.csv')
Y = pd.read_csv('target7_02.csv')

In [3]:
categorical.dtypes

Unnamed: 0      int64
STATE          object
CLUSTER         int64
HOMEOWNR       object
GENDER         object
DATASRCE        int64
SOLIH         float64
VETERANS       object
RFA_2R         object
RFA_2A         object
GEOCODE2       object
DOMAIN_A       object
DOMAIN_B        int64
dtype: object

In [4]:
X = pd.concat([numerical, categorical], axis=1).drop(columns='Unnamed: 0', axis=0)
X

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,8901,0,3712,60.000000,2.0,4.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,9702,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,4,39.0,IL,36,U,F,4,NaN,N,L,E,C,T,2
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,9702,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,9504.0,18.0,15.666667,148535,0,2,1.0,CA,14,H,M,3,NaN,N,L,G,A,S,1
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78

In [5]:
# I am checking the remaining NaN values
na = pd.DataFrame(X.isna().sum(), columns=['NA_status'])
na = na[na['NA_status']!=0]
na

,NA_status
NEXTDATE,9973
SOLIH,89212


In [6]:
# I will drop next date because I already modelled the timelag
X = X.drop(columns=['NEXTDATE'], axis=1)
X

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,8901,0,3712,60.000000,2.0,4.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,9702,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,4.0,7.741935,95515,0,4,39.0,IL,36,U,F,4,NaN,N,L,E,C,T,2
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,9702,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,18.0,15.666667,148535,0,2,1.0,CA,14,H,M,3,NaN,N,L,G,A,S,1
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30

In [7]:
# From the description of the SOLIH one can see that no answer means it can be mailed without restrictions
# I will create a new category 100 for this situation

# X['SOLIH'] = X['SOLIH'].fillna('no_limit')
X['SOLIH'].unique()

array([nan, 12.,  0.,  2.,  1.,  4.,  6.,  3.])

In [8]:
X['SOLIH'] = X['SOLIH'].fillna('no_limit')
X['SOLIH'] = X['SOLIH'].replace(12,'12_times')
X['SOLIH'] = X['SOLIH'].replace(0,'not_send')
X['SOLIH'] = X['SOLIH'].replace(2,'2_times')
X['SOLIH'] = X['SOLIH'].replace(1,'1_time')
X['SOLIH'] = X['SOLIH'].replace(4,'4_times')
X['SOLIH'] = X['SOLIH'].replace(3,'3_times')
X['SOLIH'] = X['SOLIH'].replace(6,'6_times')

X

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,8901,0,3712,60.000000,2.0,4.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,9702,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,4.0,7.741935,95515,0,4,39.0,IL,36,U,F,4,no_limit,N,L,E,C,T,2
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,9702,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,18.0,15.666667,148535,0,2,1.0,CA,14,H,M,3,no_limit,N,L,G,A,S,1
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,3

In [9]:
X['SOLIH'].unique()

array(['no_limit', '12_times', 'not_send', '2_times', '1_time', '4_times',
       '6_times', '3_times'], dtype=object)

In [13]:
nan = pd.DataFrame(X.isna().sum(), columns=['NaN_status'])
nan = nan[nan['NaN_status']!=0]
nan

,NaN_status


In [23]:
X['DOMAIN_B'].unique()

array([2, 1, 3, 4], dtype=int64)

In [27]:
#A few other columns still need changing of data type
X['DOMAIN_B'] = X['DOMAIN_B'].astype(object)
X['WEALTH1'] = X['WEALTH1'].astype(object)
X['WEALTH2'] = X['WEALTH2'].astype(object)

In [30]:
X.to_csv('X.csv')

# Train-test split, transformation and encoding

In [31]:
# For y I will first use only target B, since the first model should check only if people will donate or not

y = Y['TARGET_B']
y

0        0
1        0
2        0
3        0
4        0
        ..
95407    0
95408    0
95409    0
95410    1
95411    0
Name: TARGET_B, Length: 95412, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

In [33]:
X_train.shape

(57247, 334)

In [34]:
X_test.shape

(38165, 334)

In [35]:
categoricals_train= X_train.select_dtypes(object)
categoricals_test= X_test.select_dtypes(object)
categoricals_train.head()

,WEALTH1,WEALTH2,STATE,HOMEOWNR,GENDER,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
57139,8.0,5.0,IN,H,M,no_limit,N,L,E,B,S,2
28478,2.0,2.0,TX,H,F,no_limit,N,L,E,A,C,3
40113,1.0,0.0,WA,U,F,no_limit,N,L,E,B,C,3
94625,6.0,8.0,TX,H,M,no_limit,N,L,F,A,S,2
38201,8.0,5.0,other,H,F,no_limit,N,L,F,C,T,1


In [36]:
categoricals_train.dtypes

WEALTH1     object
WEALTH2     object
STATE       object
HOMEOWNR    object
GENDER      object
SOLIH       object
VETERANS    object
RFA_2R      object
RFA_2A      object
GEOCODE2    object
DOMAIN_A    object
DOMAIN_B    object
dtype: object

In [37]:
encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(categoricals_train)
categoricals_train_encoded = encoder.transform(categoricals_train).toarray()
categoricals_test_encoded = encoder.transform(categoricals_test).toarray()

In [38]:
encoder.categories_

[array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], dtype=object),
 array(['CA', 'FL', 'GA', 'IL', 'IN', 'MI', 'MO', 'NC', 'TX', 'WA', 'WI',
        'other'], dtype=object),
 array(['H', 'U'], dtype=object),
 array(['F', 'M', 'other'], dtype=object),
 array(['12_times', '1_time', '2_times', '3_times', '4_times', '6_times',
        'no_limit', 'not_send'], dtype=object),
 array(['N', 'Y'], dtype=object),
 array(['L'], dtype=object),
 array(['D', 'E', 'F', 'G'], dtype=object),
 array(['A', 'B', 'C', 'D'], dtype=object),
 array(['C', 'R', 'S', 'T', 'U'], dtype=object),
 array([1, 2, 3, 4], dtype=object)]

In [39]:
numericals_train = X_train.select_dtypes(np.number)
numericals_test = X_test.select_dtypes(np.number)
numericals_train

,ODATEDW,TCODE,DOB,AGE,INCOME,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE
57139,9501,1,4701,51.000000,5.0,10,0,27,17,39,7,1,3,1095,329,400,99,0,0,50,50,99,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,38,46,48,39,48,50,23,32,42,26,9,19,18,18,16,13,7,11,15,30,27,17,27,8,26,16,38,47,28,10,3,69,7,6,18,192,273,96,93,3,3,0,0,0,0,0,672,684,4,3,88,12,99,1,0,33,82,76,29,90,10,4,1,3,9,12,2,19,60,21,0,0,0,0,0,4,28,92,0,0,63,56,60,15,4,3,0,0,10,3,0,0,7,23,70,84,3480.0,83.0,527.0,438,455,439,484,16948,15,8,12,30,27,4,0,3,0,6,7,12,37,29,5,0,3,0,29,3,55,1,36,64,1,88,8,1,0,0,1,0,0,3,0,5,20,23,1,81,73,77,69,75,67,65,62,99,99,0,16,5,3,9,19,0,1,9,3,16,13,6,0,1,0,6,32,7,0,5,8,5,4,3,0,9,4,10,5,7,1,3,3,77,9,0,120,6,13,46,15,8,12,0,3,13,1,12,3,0,0,0,13,27,0,17,30,39,6,1,7,0,14,0,0,9,0,0,0,0,0,2,1,0,1,70,96,1,0,2,96,86,39,11,32,0,0,0,4,48,52,0,0,85,0,6,9,0,0,0,99,2,99,99,6,2,11,5,8,9702,22,4,12,23.0,4,2,3.0,9603,10.0,9512,3.0,9603,9412,12.0,5.750000,60484,1,3,22.0,16,3
28478,8601,2,2203,76.000000,5.0,11,3,52,31,51,6,0,2,797,220,354,99,0,0,50,50,97,0,1,2,5,0,0,0,0,0,1,0,4,0,0,1,36,45,49,39,48,51,21,46,41,14,10,23,15,13,15,16,8,18,19,34,14,15,33,15,31,34,34,31,17,7,2,55,17,9,18,145,224,61,59,6,8,2,0,0,0,0,430,447,3,3,73,27,82,18,13,26,62,49,18,79,21,8,2,6,21,22,3,18,54,24,0,0,0,0,1,1,5,34,0,11,23,45,44,12,4,8,0,20,15,7,0,4,1,15,52,76,2800.0,109.0,623.0,206,215,234,235,10434,33,32,18,7,9,1,0,0,0,30,38,19,6,4,2,0,0,0,37,9,30,18,62,38,15,78,19,0,0,0,0,3,0,0,0,12,19,26,6,76,56,68,44,68,33,40,48,65,99,4,0,14,0,3,29,0,3,11,0,26,6,8,0,0,0,16,20,11,0,0,22,7,8,8,0,3,0,0,5,6,0,2,9,83,0,0,120,10,35,30,19,1,4,0,0,19,4,14,1,2,3,2,25,52,0,31,30,51,8,4,2,0,10,0,0,10,3,0,2,0,0,0,0,0,1,64,96,4,1,0,93,37,7,8,36,0,10,10,20,28,72,0,0,78,10,12,0,0,0,89,11,85,99,91,5,2,5,10,20,9702,44,2,5,104.0,14,8,2.0,8711,14.0,9504,14.0,9504,8703,8.0,7.428571,119470,0,1,49.0,30,3
40113,8601,0,0,61.611649,1.0,0,2,33,15,22,16,3,2,9672,1592,6018,99,0,0,49,51,92,3,2,2,2,0,1,0,0,1,0,0,1,0,0,1,33,37,43,39,43,49,12,52,29,19,20,25,16,10,8,10,11,24,19,24,14,19,22,17,22,63,26,11,5,2,1,26,24,9,41,79,155,12,11,4,87,83,73,4,2,2,606,760,2,2,14,86,90,10,4,11,26,17,5,41,59,6,1,5,39,37,10,8,63,21,1,2,0,4,8,15,32,68,2,29,11,35,35,10,4,36,51,0,3,13,69,0,4,7,21,62,7840.0,337.0,881.0,120,214,185,290,12044,59,17,10,7,4,1,0,0,0,38,17,12,16,12

In [40]:
transformer = StandardScaler().fit(numericals_train)
numericals_train_standardized = transformer.transform(numericals_train)
numericals_test_standardized = transformer.transform(numericals_test)

In [41]:
pd.DataFrame(numericals_train_standardized).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321
0,1.044726,-0.056896,0.927990,-0.734719,0.705428,0.731059,-0.207504,-0.302344,-0.839612,0.356745,0.034694,-0.693797,-0.029073,-0.379420,-0.368526,-0.390644,0.852712,-0.435989,-0.653196,0.320303,-0.161709,0.672020,-0.447338,-0.222848,-0.409136,-0.467763,-0.325731,-0.176390,-0.244711,-0.241100,-0.252701,-0.198183,-0.060189,-0.452414,-0.240712,-0.103894,-0.449645,0.419218,0.490694,0.352137,0.420790,0.470033,0.303116,-0.201086,-0.920095,0.569368,0.846482,-0.516612,-0.405148,-0.421699,0.908905,1.003031,0.404187,-0.102539,-1.008688,-0.304727,-0.300554,1.582956,0.298479,0.178946,-0.185237,0.279821,-0.578581,0.529421,0.295999,0.215438,-0.073152,-0.234300,0.836642,-0.817595,-0.291697,-0.550279,0.134648,0.266752,1.036433,1.021637,0.018492,-0.690212,-0.730315,-0.639215,-0.300393,-0.274626,-0.152775,-0.412976,-0.473078,-0.096141,-0.389379,0.842115,-0.833901,0.695625,-0.822822,-0.655192,-0.203028,0.691878,1.059728,0.101432,0.569709,-0.598524,-0.730838,-0.557215,-0.693103,-0.647495,-0.734904,-0.674423,0.325250,0.635450,0.292092,-0.431950,-0.453648,-0.287637,-0.510022,-0.659808,-0.847821,-0.628346,0.612205,-0.356442,-0.642035,0.802262,0.283648,0.595165,0.336582,-0.332089,-0.557981,-0.543070,-0.593441,-0.149020,-0.298228,-0.616988,-0.533552,-0.688298,-0.554412,0.315464,0.309364,-0.015515,-0.762469,-1.167120,0.597919,0.388857,0.317638,0.307933,0.141533,-0.446285,-1.115901,-0.593315,1.560107,1.136460,-0.274033,-0.660403,1.056493,-0.408736,-0.711255,-0.973134,-0.578410,1.925646,0.968357,-0.239502,-0.664480,0.794665,-0.414304,0.157530,-0.497973,0.700255,-0.974228,-0.770807,0.824339,-1.098145,0.893696,-0.743334,-0.248805,-0.399199,-0.20676,1.209582,-0.337599,-0.732347,-0.071900,-0.364348,-0.695088,0.078847,-0.107309,-0.806591,1.201575,0.622564,0.217544,0.943616,0.348119,1.020420,0.053841,0.761813,0.940256,0.811357,-0.716959,0.229935,-1.061404,-0.209409,-0.615512,0.607617,-0.416326,-0.344126,-0.255652,0.025513,0.794123,1.145263,0.619316,-1.257480,-0.388065,-0.291502,-0.096657,1.480897,0.853014,-1.159730,0.201976,-1.442192,-0.398578,-0.216283,-0.089430,-0.686733,0.208966,-0.747861,0.804814,0.097974,0.000387,-0.720318,-0.048617,-0.924416,0.604083,0.661336,-0.464593,-0.448896,-0.334972,0.021691,1.684901,-0.810606,0.427860,-0.212599,-1.042570,-0.189706,-1.162797,-0.644482,-0.803368,-0.658998,-0.185623,-0.214881,-0.128706,-0.487934,-0.407409,-0.778391,-0.934181,1.250586,0.326582,-0.498233,0.138821,0.359916,-0.634660,0.456228,-0.276522,-0.333762,1.248520,-0.723181,-0.355589,-0.458469,-0.193266,-0.279212,1.32493,0.225031,-0.195387,-0.582166,0.594766,0.493621,-0.412424,-0.383236,-0.323653,0.337038,1.362554,1.749144,1.029080,0.649511,-0.432783,-0.731902,-0.949319,-1.310759,-0.453366,0.509004,-0.358192,-0.387307,0.930998,-0.570571,-0.850164,0.451401,-0.139344,-0.508643,-2.903885,3.144670,-1.966919,0.14

In [42]:
y_train

57139    0
28478    0
40113    0
94625    0
38201    0
        ..
65615    0
77655    0
79683    0
56088    0
38408    0
Name: TARGET_B, Length: 57247, dtype: int64

In [43]:
X_train_transformed = np.concatenate((numericals_train_standardized,categoricals_train_encoded),axis=1)
X_test_transformed = np.concatenate((numericals_test_standardized,categoricals_test_encoded),axis=1)

In [44]:
pd.DataFrame(X_train_transformed)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374
0,1.044726,-0.056896,0.927990,-0.734719,0.705428,0.731059,-0.207504,-0.302344,-0.839612,0.356745,0.034694,-0.693797,-0.029073,-0.379420,-0.368526,-0.390644,0.852712,-0.435989,-0.653196,0.320303,-0.161709,0.672020,-0.447338,-0.222848,-0.409136,-0.467763,-0.325731,-0.176390,-0.244711,-0.241100,-0.252701,-0.198183,-0.060189,-0.452414,-0.240712,-0.103894,-0.449645,0.419218,0.490694,0.352137,0.420790,0.470033,0.303116,-0.201086,-0.920095,0.569368,0.846482,-0.516612,-0.405148,-0.421699,0.908905,1.003031,0.404187,-0.102539,-1.008688,-0.304727,-0.300554,1.582956,0.298479,0.178946,-0.185237,0.279821,-0.578581,0.529421,0.295999,0.215438,-0.073152,-0.234300,0.836642,-0.817595,-0.291697,-0.550279,0.134648,0.266752,1.036433,1.021637,0.018492,-0.690212,-0.730315,-0.639215,-0.300393,-0.274626,-0.152775,-0.412976,-0.473078,-0.096141,-0.389379,0.842115,-0.833901,0.695625,-0.822822,-0.655192,-0.203028,0.691878,1.059728,0.101432,0.569709,-0.598524,-0.730838,-0.557215,-0.693103,-0.647495,-0.734904,-0.674423,0.325250,0.635450,0.292092,-0.431950,-0.453648,-0.287637,-0.510022,-0.659808,-0.847821,-0.628346,0.612205,-0.356442,-0.642035,0.802262,0.283648,0.595165,0.336582,-0.332089,-0.557981,-0.543070,-0.593441,-0.149020,-0.298228,-0.616988,-0.533552,-0.688298,-0.554412,0.315464,0.309364,-0.015515,-0.762469,-1.167120,0.597919,0.388857,0.317638,0.307933,0.141533,-0.446285,-1.115901,-0.593315,1.560107,1.136460,-0.274033,-0.660403,1.056493,-0.408736,-0.711255,-0.973134,-0.578410,1.925646,0.968357,-0.239502,-0.664480,0.794665,-0.414304,0.157530,-0.497973,0.700255,-0.974228,-0.770807,0.824339,-1.098145,0.893696,-0.743334,-0.248805,-0.399199,-0.206760,1.209582,-0.337599,-0.732347,-0.071900,-0.364348,-0.695088,0.078847,-0.107309,-0.806591,1.201575,0.622564,0.217544,0.943616,0.348119,1.020420,0.053841,0.761813,0.940256,0.811357,-0.716959,0.229935,-1.061404,-0.209409,-0.615512,0.607617,-0.416326,-0.344126,-0.255652,0.025513,0.794123,1.145263,0.619316,-1.257480,-0.388065,-0.291502,-0.096657,1.480897,0.853014,-1.159730,0.201976,-1.442192,-0.398578,-0.216283,-0.089430,-0.686733,0.208966,-0.747861,0.804814,0.097974,0.000387,-0.720318,-0.048617,-0.924416,0.604083,0.661336,-0.464593,-0.448896,-0.334972,0.021691,1.684901,-0.810606,0.427860,-0.212599,-1.042570,-0.189706,-1.162797,-0.644482,-0.803368,-0.658998,-0.185623,-0.214881,-0.128706,-0.487934,-0.407409,-0.778391,-0.934181,1.250586,0.326582,-0.498233,0.138821,0.359916,-0.634660,0.456228,-0.276522,-0.333762,1.248520,-0.723181,-0.355589,-0.458469,-0.193266,-0.279212,1.32493,0.225031,-0.195387,-0.582166,0.594766,0.493621,-0.412424,-0.383236,-0.323653,0.

## Logistic Regression Model

In [45]:
%%time
classification = LogisticRegression(random_state=0, solver='saga', max_iter=1000, multi_class='multinomial').fit(X_train_transformed, y_train)

CPU times: total: 7min 33s
Wall time: 7min 37s


C:\Users\luana\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [46]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.9491942879601729

In [47]:
# Target is very unbalanced, therefore score so high:
# If the model predicts that everyone will NOT donate, it will still get it right many times

confusion_matrix(y_test, predictions)

array([[36226,     0],
       [ 1939,     0]], dtype=int64)

## Managing imbalance in the dataset

In [63]:
y_train.value_counts()

0    54343
1     2904
Name: TARGET_B, dtype: int64

In [62]:
all_data = pd.DataFrame(X_train_transformed)
all_data = pd.concat([all_data, y_train.reset_index(drop=True)], axis=1)
all_data
all_data['TARGET_B'].value_counts(dropna=False)

0    54343
1     2904
Name: TARGET_B, dtype: int64

In [64]:
category_0 = all_data[all_data['TARGET_B'] == 0]
category_1 = all_data[all_data['TARGET_B'] == 1]

### Downsampling

In [81]:
category_0_undersampled = resample(category_0, replace=False, n_samples = len(category_1))

In [82]:
data_undersampled = pd.concat([category_0_undersampled, category_1], axis=0)
data_undersampled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,TARGET_B
45113,-0.700713,-0.056896,-0.432065,1.269963,1.238426,-0.361301,1.163640,0.745496,-0.507545,0.243953,-0.872018,-0.304168,1.891954,-0.222362,-0.243656,-0.055496,0.852712,-0.435989,-0.653196,-0.035198,0.181232,0.102212,0.030695,-0.222848,-0.126801,0.479477,-0.325731,-0.176390,-0.244711,-0.241100,0.763838,-0.198183,-0.060189,0.614873,0.556192,-0.103894,-0.153484,-0.176408,-0.592692,-0.627367,0.010739,-0.527503,-0.561059,-1.127984,1.629235,-1.303698,-0.682640,0.131177,1.254083,0.058827,-0.471354,-0.445342,-0.258950,-0.545607,2.310267,-0.035440,-1.034761,-0.596890,-0.391405,-0.808109,-0.452512,-0.640439,2.144865,0.044791,-1.698230,-1.594180,-1.325613,-0.759209,-0.623148,1.626872,-0.495256,0.455623,-1.582138,-1.572190,-1.927935,-1.749399,-0.544812,2.457954,2.718558,2.933008,-0.300393,-0.274626,-0.152775,0.943243,1.034348,0.336851,0.501700,-2.153079,2.282444,-0.766424,1.093421,-0.464604,-1.274773,-1.681838,-1.338940,-1.318603,-1.121638,1.389957,-0.163824,-0.557215,-0.051876,1.610966,1.288050,0.283140,-0.835862,0.704126,-0.192809,0.006355,0.041696,-0.287637,1.185418,2.136111,1.718128,1.303047,0.848246,0.032100,0.155432,-1.154380,-1.015680,-0.935747,-1.605160,-1.181518,0.774041,2.839894,-0.593441,-1.169133,0.236341,3.277436,-0.533552,0.274983,0.561875,0.649580,0.925995,1.296755,0.608831,-0.192523,0.291933,0.745104,0.756410,1.272088,1.819554,-0.515000,-0.363054,0.044876,-0.044069,0.210110,0.409968,0.532651,-0.480796,1.530081,-0.224566,-1.082111,-0.052740,-0.894465,0.031343,0.208733,2.142240,-0.475865,2.299161,-0.845839,-0.818883,0.875702,-0.374642,2.358923,-2.291393,1.824930,0.744069,-0.891937,-0.248805,-0.180119,-0.206760,-0.142277,-0.337599,-0.194053,-0.071900,-0.364348,-0.800082,-0.364329,-0.559198,-0.619053,-0.115909,0.695692,0.963298,0.435763,0.745785,0.657588,0.905425,0.472958,-0.097236,0.811357,-0.176124,2.295671,0.753347,-1.371416,0.830150,-0.612211,-0.416326,-0.344126,-0.420881,-0.558626,-0.574669,-1.073828,-0.931452,-0.250054,-0.388065,-0.291502,0.637011,-1.077607,-0.716093,-0.303977,0.839678,-0.635040,0.894460,-0.216283,-0.089430,-0.686733,2.871183,-0.008131,0.576172,-0.630748,-0.933523,-0.327648,1.851649,-0.009686,0.118935,-0.478283,-0.464593,1.791623,-0.697708,-1.159916,-1.436391,0.171114,-0.452902,1.428547,2.354622,0.842059,-1.546470,-0.644482,-1.273061,0.031977,1.949162,1.177051,5.026149,1.029830,0.739019,1.646541,-0.589014,-0.605732,0.209762,2.298020,-0.322387,0.359916,0.099224,0.096394,-0.276522,-0.333762,0.046144,0.076215,0.096665,-0.116502,-0.193266,-0.279212,0.28540,1.026521,-0.195387,0.562100,-0.733894,-0.347132,0.502675,0.442692,-0.323653,0.4

In [83]:
X_undersampled = data_undersampled.drop(columns='TARGET_B',axis=1)
X_undersampled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374
45113,-0.700713,-0.056896,-0.432065,1.269963,1.238426,-0.361301,1.163640,0.745496,-0.507545,0.243953,-0.872018,-0.304168,1.891954,-0.222362,-0.243656,-0.055496,0.852712,-0.435989,-0.653196,-0.035198,0.181232,0.102212,0.030695,-0.222848,-0.126801,0.479477,-0.325731,-0.176390,-0.244711,-0.241100,0.763838,-0.198183,-0.060189,0.614873,0.556192,-0.103894,-0.153484,-0.176408,-0.592692,-0.627367,0.010739,-0.527503,-0.561059,-1.127984,1.629235,-1.303698,-0.682640,0.131177,1.254083,0.058827,-0.471354,-0.445342,-0.258950,-0.545607,2.310267,-0.035440,-1.034761,-0.596890,-0.391405,-0.808109,-0.452512,-0.640439,2.144865,0.044791,-1.698230,-1.594180,-1.325613,-0.759209,-0.623148,1.626872,-0.495256,0.455623,-1.582138,-1.572190,-1.927935,-1.749399,-0.544812,2.457954,2.718558,2.933008,-0.300393,-0.274626,-0.152775,0.943243,1.034348,0.336851,0.501700,-2.153079,2.282444,-0.766424,1.093421,-0.464604,-1.274773,-1.681838,-1.338940,-1.318603,-1.121638,1.389957,-0.163824,-0.557215,-0.051876,1.610966,1.288050,0.283140,-0.835862,0.704126,-0.192809,0.006355,0.041696,-0.287637,1.185418,2.136111,1.718128,1.303047,0.848246,0.032100,0.155432,-1.154380,-1.015680,-0.935747,-1.605160,-1.181518,0.774041,2.839894,-0.593441,-1.169133,0.236341,3.277436,-0.533552,0.274983,0.561875,0.649580,0.925995,1.296755,0.608831,-0.192523,0.291933,0.745104,0.756410,1.272088,1.819554,-0.515000,-0.363054,0.044876,-0.044069,0.210110,0.409968,0.532651,-0.480796,1.530081,-0.224566,-1.082111,-0.052740,-0.894465,0.031343,0.208733,2.142240,-0.475865,2.299161,-0.845839,-0.818883,0.875702,-0.374642,2.358923,-2.291393,1.824930,0.744069,-0.891937,-0.248805,-0.180119,-0.206760,-0.142277,-0.337599,-0.194053,-0.071900,-0.364348,-0.800082,-0.364329,-0.559198,-0.619053,-0.115909,0.695692,0.963298,0.435763,0.745785,0.657588,0.905425,0.472958,-0.097236,0.811357,-0.176124,2.295671,0.753347,-1.371416,0.830150,-0.612211,-0.416326,-0.344126,-0.420881,-0.558626,-0.574669,-1.073828,-0.931452,-0.250054,-0.388065,-0.291502,0.637011,-1.077607,-0.716093,-0.303977,0.839678,-0.635040,0.894460,-0.216283,-0.089430,-0.686733,2.871183,-0.008131,0.576172,-0.630748,-0.933523,-0.327648,1.851649,-0.009686,0.118935,-0.478283,-0.464593,1.791623,-0.697708,-1.159916,-1.436391,0.171114,-0.452902,1.428547,2.354622,0.842059,-1.546470,-0.644482,-1.273061,0.031977,1.949162,1.177051,5.026149,1.029830,0.739019,1.646541,-0.589014,-0.605732,0.209762,2.298020,-0.322387,0.359916,0.099224,0.096394,-0.276522,-0.333762,0.046144,0.076215,0.096665,-0.116502,-0.193266,-0.279212,0.28540,1.026521,-0.195387,0.562100,-0.733894,-0.347132,0.502675,0.442692,-0.323653,0.416517,-1.

In [85]:
y_undersampled = data_undersampled['TARGET_B']
y_undersampled

45113    0
35507    0
34011    0
36613    0
30303    0
        ..
57198    1
57200    1
57223    1
57226    1
57231    1
Name: TARGET_B, Length: 5808, dtype: int64

In [86]:
%%time
classification_undersampled = LogisticRegression(random_state=0, solver='saga', max_iter=1000,
                                                 multi_class='multinomial').fit(X_undersampled, y_undersampled)

CPU times: total: 51.6 s
Wall time: 52.3 s


C:\Users\luana\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [89]:
predictions_undersampled = classification_undersampled.predict(X_test_transformed)
classification_undersampled.score(X_test_transformed, y_test)

0.5675094982313638

In [90]:
confusion_matrix(y_test, predictions_undersampled)

array([[20622, 15604],
       [  902,  1037]], dtype=int64)

In [ ]:
# Now the model can at least predict some donators....

### Upsampling (oversampling)

In [79]:
category_1_oversampled = resample(category_1,replace=True, n_samples = len(category_0))

In [91]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [92]:
X_upsampled = data_upsampled.drop(columns='TARGET_B',axis=1)
X_upsampled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374
0,1.044726,-0.056896,0.927990,-0.734719,0.705428,0.731059,-0.207504,-0.302344,-0.839612,0.356745,0.034694,-0.693797,-0.029073,-0.379420,-0.368526,-0.390644,0.852712,-0.435989,-0.653196,0.320303,-0.161709,0.672020,-0.447338,-0.222848,-0.409136,-0.467763,-0.325731,-0.176390,-0.244711,-0.241100,-0.252701,-0.198183,-0.060189,-0.452414,-0.240712,-0.103894,-0.449645,0.419218,0.490694,0.352137,0.420790,0.470033,0.303116,-0.201086,-0.920095,0.569368,0.846482,-0.516612,-0.405148,-0.421699,0.908905,1.003031,0.404187,-0.102539,-1.008688,-0.304727,-0.300554,1.582956,0.298479,0.178946,-0.185237,0.279821,-0.578581,0.529421,0.295999,0.215438,-0.073152,-0.234300,0.836642,-0.817595,-0.291697,-0.550279,0.134648,0.266752,1.036433,1.021637,0.018492,-0.690212,-0.730315,-0.639215,-0.300393,-0.274626,-0.152775,-0.412976,-0.473078,-0.096141,-0.389379,0.842115,-0.833901,0.695625,-0.822822,-0.655192,-0.203028,0.691878,1.059728,0.101432,0.569709,-0.598524,-0.730838,-0.557215,-0.693103,-0.647495,-0.734904,-0.674423,0.325250,0.635450,0.292092,-0.431950,-0.453648,-0.287637,-0.510022,-0.659808,-0.847821,-0.628346,0.612205,-0.356442,-0.642035,0.802262,0.283648,0.595165,0.336582,-0.332089,-0.557981,-0.543070,-0.593441,-0.149020,-0.298228,-0.616988,-0.533552,-0.688298,-0.554412,0.315464,0.309364,-0.015515,-0.762469,-1.167120,0.597919,0.388857,0.317638,0.307933,0.141533,-0.446285,-1.115901,-0.593315,1.560107,1.136460,-0.274033,-0.660403,1.056493,-0.408736,-0.711255,-0.973134,-0.578410,1.925646,0.968357,-0.239502,-0.664480,0.794665,-0.414304,0.157530,-0.497973,0.700255,-0.974228,-0.770807,0.824339,-1.098145,0.893696,-0.743334,-0.248805,-0.399199,-0.206760,1.209582,-0.337599,-0.732347,-0.071900,-0.364348,-0.695088,0.078847,-0.107309,-0.806591,1.201575,0.622564,0.217544,0.943616,0.348119,1.020420,0.053841,0.761813,0.940256,0.811357,-0.716959,0.229935,-1.061404,-0.209409,-0.615512,0.607617,-0.416326,-0.344126,-0.255652,0.025513,0.794123,1.145263,0.619316,-1.257480,-0.388065,-0.291502,-0.096657,1.480897,0.853014,-1.159730,0.201976,-1.442192,-0.398578,-0.216283,-0.089430,-0.686733,0.208966,-0.747861,0.804814,0.097974,0.000387,-0.720318,-0.048617,-0.924416,0.604083,0.661336,-0.464593,-0.448896,-0.334972,0.021691,1.684901,-0.810606,0.427860,-0.212599,-1.042570,-0.189706,-1.162797,-0.644482,-0.803368,-0.658998,-0.185623,-0.214881,-0.128706,-0.487934,-0.407409,-0.778391,-0.934181,1.250586,0.326582,-0.498233,0.138821,0.359916,-0.634660,0.456228,-0.276522,-0.333762,1.248520,-0.723181,-0.355589,-0.458469,-0.193266,-0.279212,1.32493,0.225031,-0.195387,-0.582166,0.594766,0.493621,-0.412424,-0.383236,-0.323653,0.

In [93]:
y_upsampled = data_upsampled['TARGET_B']
y_upsampled

0        0
1        0
2        0
3        0
4        0
        ..
22702    1
43985    1
52576    1
34340    1
20891    1
Name: TARGET_B, Length: 108686, dtype: int64

In [94]:
%%time
classification_upsampled = LogisticRegression(random_state=0, solver='saga', max_iter=1000,
                                                 multi_class='multinomial').fit(X_upsampled, y_upsampled)

CPU times: total: 12min 48s
Wall time: 12min 52s


C:\Users\luana\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [98]:
predictions_upsampled = classification_upsampled.predict(X_test_transformed)
classification_upsampled.score(X_test_transformed, y_test)

0.6169789073758679

In [96]:
confusion_matrix(y_test, predictions_upsampled)

array([[22599, 13627],
       [  991,   948]], dtype=int64)

In [97]:
confusion_matrix(y_test, predictions_undersampled)

array([[20622, 15604],
       [  902,  1037]], dtype=int64)

In [ ]:
## The upsampled dataset delivered the best results.